In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/df_segmentation.csv", index_col=[0])
df["date_mensuelle"] = pd.to_datetime(df["date_mensuelle"])
df

,date_mensuelle,TARGET,DAYS_CREDIT_ENDDATE_disc_int,DAYS_FIRST_DRAWING_disc_int,RATE_DOWN_PAYMENT_disc_int,AMT_PAYMENT_disc_int,NAME_INCOME_TYPE_discret,NAME_EDUCATION_TYPE_discret,NAME_FAMILY_STATUS_discret,OCCUPATION_TYPE_discret,...,FLAG_WORK_PHONE,FLAG_PHONE,LIVE_CITY_NOT_WORK_CITY,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CODE_GENDER,Score_ind,date_trimestrielle,Classes
0,2013-01-01,0,[0.0;214193.0],[730486.0;9131075.0],[-0.0;0.2],[150217.24;25537053.78],high_income,lower,couple,low_skilled,...,zero,un,zero,Cash loans,Y,Y,M,80.851064,2013_1,1
1,2013-01-01,0,[-155271.0;-1.0],[730486.0;9131075.0],[-0.0;0.2],[150217.24;25537053.78],high_income,higher,couple,low_skilled,...,zero,un,zero,Cash loans,N,Y,F,276.595745,2013_1,2
2,2013-01-01,0,[0.0;214193.0],[-3505.0;730484.0],[-0.0;0.2],[0.0;150215.44],high_income,lower,couple,low_skilled,...,un,zero,zero,Cash loans,Y,N,F,200.000000,2013_1,2
3,2013-01-01,0,[-155271.0;-1.0],[-3505.0;730484.0],[-0.0;0.2],[0.0;150215.44],high_income,lower,alone,low_skilled,...,zero,zero,zero,Cash loans,Y,Y,F,395.744681,2013_1,3
4,2013-01-01,0,[0.0;214193.0],[-3505.0;730484.0],[-0.0;0.2],[0.0;150215.44],high_income,lower,alone,low_skilled,...,zero,un,un,Cash loans,N,N,F,200.000000,2013_1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306102,2020-12-01,0,[0.0;214193.0],[-3505.0;730484.0],[-0.0;0.2],[0.0;150215.44],Low_income,lower,couple,low_skilled,...,zero,zero,zero,Cash loans,N,N,F,234.042553,2020_4,2
306103,2020-12-01,0,[-155271.0;-1.0],[730486.0;9131075.0],[0.2;4.48],[150217.24;25537053.78],high_income,lower,alone,low_skilled,...,zero,zero,un,Cash loans,N,Y,F,582.978723,2020_4,5
306104,2020-12-01,0,[0.0;214193.0],[-3505.0;730484.0],[-0.0;0.2],[0.0;150215.44],high_income,lower,couple,low_skilled,...,zero,un,zero,Cash loans,N,Y,F,200.000000,2020_4,2
306105,2020-12-01,0,[0.0;214193.0],[-3505.0;730484.0],[-0.0;0.2],[0.0;150215.44],Low_income,lower,alone,low_skilled,...,zero,zero,zero,Cash loans,N,Y,M,234.042553,2020_4,2


# MOC C

In [12]:
resultats = df.groupby("Classes").agg(moyenne_TARGET=("TARGET", "mean")).to_dict()["moyenne_TARGET"]
# convertit le résultat de l'opération groupby en un dictionnaire 
# les clés sont les valeurs uniques de la colonne "Classes" et les valeurs sont les moyennes calculées pour chaque groupe. 
# L'index "moyenne_TARGET" est utilisé pour accéder aux valeurs de moyenne dans le dictionnaire résultant.

resultats

{1: 0.17708784076157508,
 2: 0.1113129079806275,
 3: 0.08011071110594975,
 4: 0.06767496258844383,
 5: 0.05113112908178934,
 6: 0.03869101428313144,
 7: 0.03712362734679419}

In [4]:
# resultats

{1: 0.17558199689601656,
 2: 0.11039856923863056,
 3: 0.0796209282499653,
 4: 0.06726292921028129,
 5: 0.050026666666666664,
 6: 0.0385176539247155,
 7: 0.03720294586591755}

In [5]:
def Boostrapping_Classes(classe):
    df_classe = df[df['Classes'] == classe]
    tx_defaut_liste = []

    for _ in range(1000):
        echantillon = df_classe["TARGET"].sample(n=len(df_classe), replace=True)
        tx_defaut_liste.append(echantillon.mean())
        
    tx_defaut_liste = np.array(tx_defaut_liste)

    decile9 = np.percentile(tx_defaut_liste, 90)

    MOC_C = decile9 - resultats[classe]
    
    return(classe, resultats[classe], decile9, MOC_C)

In [6]:
MOC_C = pd.DataFrame(columns = ["Classe", "LRA", "Décile9", "Moc_C"])

In [7]:
for i in range(1,8):
    classe, LRA, decile9, MOC_C_classe = Boostrapping_Classes(i)
    MOC_C.loc[len(MOC_C)] = [classe, LRA, decile9, MOC_C_classe]

In [8]:
MOC_C

,Classe,LRA,Décile9,Moc_C
0,1.0,0.175582,0.180341,0.004759
1,2.0,0.110399,0.111884,0.001486
2,3.0,0.079621,0.081227,0.001606
3,4.0,0.067263,0.068406,0.001143
4,5.0,0.050027,0.051200,0.001173
5,6.0,0.038518,0.040050,0.001532
6,7.0,0.037203,0.039329,0.002126


# MOC A
### Valeurs manquantes dans les variables explicatives

In [3]:
var_explicatives = ['REGION_RATING_CLIENT_W_CITY', 'DAYS_CREDIT_ENDDATE', 'RATE_DOWN_PAYMENT', 'AMT_PAYMENT', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE']
len(var_explicatives)

6

In [4]:
train = pd.read_csv("../data/application_train_vf.csv")
df_bur = pd.read_csv('../data/bureau.csv')
df_bur_group = df_bur[['DAYS_CREDIT_ENDDATE', 'SK_ID_CURR']].groupby('SK_ID_CURR').sum()
df_bur_group.reset_index(inplace=True)

df_prev = pd.read_csv('../data/previous_application.csv')
df_prev_group = df_prev[['SK_ID_CURR', 'DAYS_FIRST_DRAWING', 'RATE_DOWN_PAYMENT']].groupby(
    'SK_ID_CURR').sum()
df_prev_group.reset_index(inplace=True)

df_ins = pd.read_csv('../data/installments_payments.csv')
df_ins_group = df_ins[['SK_ID_CURR', 'AMT_PAYMENT']].groupby('SK_ID_CURR').sum()
df_ins_group.reset_index(inplace=True)

train = train.merge(df_bur_group, on='SK_ID_CURR', how='left')
train = train.merge(df_prev_group, on='SK_ID_CURR', how='left')
train = train.merge(df_ins_group, on='SK_ID_CURR', how='left')

In [5]:
train[var_explicatives].isna().sum()

REGION_RATING_CLIENT_W_CITY        0
DAYS_CREDIT_ENDDATE            43764
RATE_DOWN_PAYMENT              16395
AMT_PAYMENT                    15810
NAME_INCOME_TYPE                   0
OCCUPATION_TYPE                96049
dtype: int64

### Période Manquante

In [6]:
max_year = df["date_mensuelle"].dt.year.max()
print(max_year)
min_year = df["date_mensuelle"].dt.year.min()
print(min_year)

2020
2013


In [7]:
# Examiner les dates de début et de fin dans vos données
date_debut = df['date_trimestrielle'].min()
date_fin = df['date_trimestrielle'].max()
print("Date de début :", date_debut)
print("Date de fin :", date_fin)

Date de début : 2013_1
Date de fin : 2020_4


In [8]:
periodes_attendues = [f"{year}_{quarter}" for year in range(min_year, max_year) for quarter in range(1, 5)]
periodes_reelles = df['date_trimestrielle'].unique()

periodes_manquantes = set(periodes_attendues) - set(periodes_reelles)
print("Périodes manquantes dans les données :", periodes_manquantes)

Périodes manquantes dans les données : set()


### Crise Covid

In [9]:
df['periode_crise_covid'] = ((df['date_trimestrielle'] == '2020_1') 
| (df['date_trimestrielle'] == '2020_2'))

sans_covid = df[df['periode_crise_covid'] == False].copy()

## Version 2 ajusté 

In [38]:
resultats_I = df.groupby("Classes").agg(
    moyenne_TARGET=("TARGET", "mean"),
    nb_individus=("TARGET", "size")
)
resultats_I['taux_individus'] = (resultats_I['nb_individus'] / df.shape[0]) * 100

resultats_I

,moyenne_TARGET,nb_individus,taux_individus
Classes,,,
1,0.177088,9244,3.019859
2,0.111313,75984,24.822693
3,0.080111,51666,16.878412
4,0.067675,68829,22.485275
5,0.051131,58614,19.148206
6,0.038691,27655,9.034423
7,0.037124,14115,4.611133


In [19]:
sum(nb_total_individus_par_classe)

306107

In [28]:
Covid_défaut = pd.DataFrame(columns = ["Classe", "Taux avec Covid", "Taux sans Covid", "Marge Conservatisme (%)"])

In [29]:
for i in range(1,8): # 7 segmentations 
    tx_ss_covid = sans_covid[sans_covid["Classes"]==i]["TARGET"].mean()
    
    marge = ((tx_ss_covid/resultats[i])-1)*100
    
    #ajouter une ligne au df "Covid_défaut" contenant les info suivant : 
    Covid_défaut.loc[len(Covid_défaut)] = [i, resultats[i], tx_ss_covid, marge]

### V1

In [30]:
Covid_défaut

,Classe,Taux avec Covid,Taux sans Covid,Marge Conservatisme (%)
0,1.0,0.177088,0.176708,-0.214633
1,2.0,0.111313,0.111050,-0.235976
2,3.0,0.080111,0.080242,0.163658
3,4.0,0.067675,0.067785,0.163052
4,5.0,0.051131,0.051119,-0.024128
5,6.0,0.038691,0.038513,-0.458967
6,7.0,0.037124,0.037096,-0.074960


### V2

In [37]:
# Calculer la somme totale des individus par classe
total_individus_par_classe = resultats_I['nb_individus']

Covid_defaut = Covid_défaut.copy()

Covid_defaut['Impact ajusté'] = 0.0

for index, row in Covid_defaut.iterrows():
    classe = row['Classe']
    # Calculer la différence entre le taux de défaut moyen avec et sans la crise COVID-19
    difference =  row['Taux sans Covid'] - row['Taux avec Covid']
    pourcentage = (difference / row['Taux avec Covid']) * 100
    
    # Ponderer la difference par le nombre d'individus dans cette classe
    poids = total_individus_par_classe.loc[classe]
    impact_pondere = pourcentage * poids / total_individus_par_classe.sum()
    
    # Affecter la valeur ajustée à la colonne correspondante
    Covid_defaut.loc[index, 'Impact ajusté'] = impact_pondere

Covid_defaut


,Classe,Taux avec Covid,Taux sans Covid,Marge Conservatisme (%),Impact ajusté
0,1.0,0.177088,0.176708,-0.214633,-0.006482
1,2.0,0.111313,0.111050,-0.235976,-0.058576
2,3.0,0.080111,0.080242,0.163658,0.027623
3,4.0,0.067675,0.067785,0.163052,0.036663
4,5.0,0.051131,0.051119,-0.024128,-0.004620
5,6.0,0.038691,0.038513,-0.458967,-0.041465
6,7.0,0.037124,0.037096,-0.074960,-0.003457
